# Mid Train 中期训练
Mid Train 会在 Pre Train 的基础上，进一步特化模型学到的内容，让其学习使用各种特殊 token、感知元数据（标题、形式、作者）并与诗词本体建立一些关联。与 Pre Train 阶段相比，会有如下变化：
1. 每一首诗词通过设定好的特殊 token 进行编码，生成一个独立的序列。
2. 每一首诗词作为一份单独的训练数据，长度的问题会通过 Padding 的方式补齐，Padding 的部分不会计算 Loss。
3. 使用更精细的训练策略，让模型准确学到一个完整诗词序列的模式。

Notebook 中的训练代码是简化过的，方便快速运行和理解。

In [1]:
import random

import torch

from nanopoet.common import CharTokenizer, PADDING
from nanopoet.dataset import load_raw_data, split_data
from nanopoet.model import GPTLanguageModel

# 初始化随机种子，让重复执行的结果稳定
random.seed(12345)
torch.manual_seed(12345)

# 首先加载数据、设备信息
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
data = load_raw_data("../raw")

# 初始化分词器
tokenizer = CharTokenizer("".join(["".join(list(d.values())) for d in data]))


In [2]:
# 初始化模型结构
block_size = 256
model = GPTLanguageModel(
    vocab_size=tokenizer.vocab_size,
    emb_size=256,
    block_size=block_size,
    layer_num=8,
    head_num=8,
    dropout=0.1,
).to(device)

# 加载 Pre Train 的训练结果
pretrain_state = torch.load("./output/02_pre_train_model.pt", map_location=device)
model.load_state_dict(pretrain_state, strict=True)

<All keys matched successfully>

In [3]:
from nanopoet.common import encode_poem

# 构建 Mid Train 阶段使用的数据集
train, val = split_data(data)

# 使用特殊 token 将每一首诗词转化成特定的序列
train_ps = [encode_poem(t) for t in train]
val_ps = [encode_poem(t) for t in val]

# 打印对比一下转换前后
print("转换前：", train[0])
print("\n转换后：", train_ps[0])

# 将转换后的序列编码成数据
train_data = [torch.tensor(tokenizer.encode(poem), dtype=torch.long) for poem in train_ps]
val_data = [torch.tensor(tokenizer.encode(poem), dtype=torch.long) for poem in train_ps]


# 制作一个数据加载函数，这里与 Pre 阶段的策略是不同的，需要注意
def get_batch(poems_encoded, batch_size, pad_token_id, block_size, device):
    # 随机抽取一个批次的诗词
    indices = torch.randint(len(poems_encoded), (batch_size,))
    batch = [poems_encoded[i] for i in indices]
    # 由于诗词数据的长度不同，但我们训练使用的上下文长度（block_size）是固定的
    # 因此需要对超长的进行截断、过短的使用 padding token 补齐
    # 补齐的部分，不需要进行 Loss 计算（毕竟我们不希望模型学会输出一堆没有意义的 Padding 字符）
    # 因此需要返回补齐的 Mask，用来遮罩 Loss
    batch = [p[:block_size] if len(p) > block_size else p for p in batch]
    # 获取批次中的最大长度
    # 可能这一批次中的数据，长度全都小于 block_size，如果是这样为了节约计算，不需要都强制补齐到block_size，只要对齐一个批次的数据长度即可
    max_len = max(len(p) for p in batch)
    # 创建填充后的批次
    batch_x = []
    batch_y = []
    batch_mask = []
    for poem in batch:
        # 输入: poem[:-1], 目标: poem[1:]
        if len(poem) > 1:
            x = poem[:-1]
            y = poem[1:]

            # 填充
            pad_len = max_len - 1 - len(x)
            if pad_len > 0:
                x = torch.cat([x, torch.full((pad_len,), pad_token_id, dtype=torch.long)])
                y = torch.cat([y, torch.full((pad_len,), pad_token_id, dtype=torch.long)])
                # mask: 真实token为1，填充为0
                mask = torch.cat(
                    [torch.ones(len(poem) - 1, dtype=torch.long), torch.zeros(pad_len, dtype=torch.long)])
            else:
                mask = torch.ones(len(x), dtype=torch.long)

            batch_x.append(x)
            batch_y.append(y)
            batch_mask.append(mask)
    return torch.stack(batch_x).to(device), torch.stack(batch_y).to(device), torch.stack(batch_mask).to(device)


转换前： {'content': '卧虹千尺界湖光。冷浸月茫茫。当日三高何处，渔唱入凄凉。人世事，纵轩裳。梦黄梁。有谁蓑笠，一钓丝风，吹尽荷香。', 'title': '一丝风诉衷情令', 'author': '张辑', 'style': '一丝风诉衷情令'}

转换后： BA张辑aS一丝风诉衷情令sT一丝风诉衷情令tC卧虹千尺界湖光。冷浸月茫茫。当日三高何处，渔唱入凄凉。人世事，纵轩裳。梦黄梁。有谁蓑笠，一钓丝风，吹尽荷香。c


In [4]:
# 设置训练配置
batch_size = 32  # 独立诗词样本用更小的batch size
max_steps = 100
eval_interval = 10
eval_iters = 5

# Mid 阶段训练我们使用更精细的学习率控制策略，让学习率能够随着训练过程的逐渐加深，而减小学习率
# 学习率调度（参考 nanochat mid-train）
learning_rate = 3e-4
init_lr_frac = 0.5  # Mid训练从预训练LR的50%开始（更保守）
warmdown_start_ratio = 0.8  # 前80%保持不变，最后20%线性衰减到0
final_lr_frac = 0.0

# 梯度裁剪
# 设定一个梯度裁剪可以防止某个 batch 导致的梯度变化大幅增加，近期模型训练因梯度爆炸导致无法收敛的情况
grad_clip = 1.0

# 初始化优化器
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=learning_rate * init_lr_frac
)


In [6]:
from nanopoet.common import BEGIN,PADDING
import torch.nn.functional as F


# 制作一个loss 计算函数，计算 masking 之后的 loss
def get_loss_with_mask(logits, yb, mask):
    # 计算loss（只计算非填充位置）
    B, T, C = logits.shape
    logits_flat = logits.view(B * T, C)
    targets_flat = yb.view(B * T)
    mask_flat = mask.view(B * T)
    # 只计算mask=1的位置
    if mask_flat.sum() > 0:
        loss = F.cross_entropy(logits_flat[mask_flat == 1], targets_flat[mask_flat == 1])
    else:
        loss = torch.tensor(0.0, device=device)
    return loss


# 简化的训练循环
pad_token_id = tokenizer.encode(PADDING)[0]
for step in range(max_steps):
    # 获取对齐后的数据以及填充 mask
    xb, yb, mask = get_batch(train_data, batch_size, pad_token_id, block_size, device)
    # 前向传播，但忽略模型直接返回的 Loss 值
    logits, _ = model(xb, yb)
    # 单独计算 loss
    loss = get_loss_with_mask(logits, yb, mask)
    # 反向传播
    optimizer.zero_grad()
    loss.backward()
    # 梯度裁剪
    if grad_clip > 0.0:
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

    # 计算学习率变化
    warmdown_start_step = int(warmdown_start_ratio * max_steps)

    if step < warmdown_start_step:
        # 第一阶段，学习率不变
        lr_mult = 1.0
    else:
        # 最后阶段，学习率随着 step 增加线性衰减
        progress = (max_steps - step) / (max_steps - warmdown_start_step)
        lr_mult = progress * 1.0 + (1 - progress) * final_lr_frac

    # 应用新的学习率
    for param_group in optimizer.param_groups:
        param_group['lr'] = learning_rate * lr_mult

    optimizer.step()

    # 定期评估、打印Loss、学习率变化，并生成一段测试样本
    if step % eval_interval == 0:
        model.eval()
        out = {}
        for split, data in [('train', train_data), ('val', val_data)]:
            losses = torch.zeros(eval_iters)
            for k in range(eval_iters):
                X,Y,M = get_batch(data, batch_size, pad_token_id, block_size, device)
                logits, _ = model(X, Y)
                losses[k] = get_loss_with_mask(logits, Y, M).item()
            out[split] = losses.mean()
        context = torch.tensor([tokenizer.encode(BEGIN)], dtype=torch.long, device=device)
        generated = model.generate(context, max_new_tokens=20)
        sample = tokenizer.decode(generated[0].tolist())
        model.train()
        current_lr = (learning_rate * init_lr_frac) * lr_mult

        print(f"Step {step:5d} | 训练Loss: {out['train']:.4f} | 验证Loss: {out['val']:.4f} | LR: {current_lr:.2e} (x{lr_mult:.3f})")
        print("生成样本：", sample)



Step     0 | 训练Loss: 7.4114 | 验证Loss: 7.4523 | LR: 1.50e-04 (x1.000)
生成样本： B業鬐羹當巧經元將是舞，战。認杖福泛迎。行
Step    10 | 训练Loss: 6.9597 | 验证Loss: 6.9636 | LR: 1.50e-04 (x1.000)
生成样本： B獿厩宴諡蘧罚築苦運來子，風樣其素韻界钜花
Step    20 | 训练Loss: 6.6421 | 验证Loss: 6.6594 | LR: 1.50e-04 (x1.000)
生成样本： B皿歸䜝散坂柧事浇。雲誤浮路迹度絲能，此。
Step    30 | 训练Loss: 6.3811 | 验证Loss: 6.3683 | LR: 1.50e-04 (x1.000)
生成样本： B侉趣恋珏報決疲醓運牛遂言，宋畫阿道兵怨虎
Step    40 | 训练Loss: 6.1743 | 验证Loss: 6.1063 | LR: 1.50e-04 (x1.000)
生成样本： B密藞霜丘搔切井巾S莎節嫣交欠渺脚舍相送處
Step    50 | 训练Loss: 6.0228 | 验证Loss: 5.9918 | LR: 1.50e-04 (x1.000)
生成样本： B嘶高锺臣駷a硬荔Ss據餉勉上与陽種徑t讨
Step    60 | 训练Loss: 5.7672 | 验证Loss: 5.8326 | LR: 1.50e-04 (x1.000)
生成样本： BA鹜a睎a绤滿書綸s辔任五素計奉玉立泉去
Step    70 | 训练Loss: 5.7084 | 验证Loss: 5.6721 | LR: 1.50e-04 (x1.000)
生成样本： B師蘼a罙蕚S韵德汗saT敲耐也游tC阳參
Step    80 | 训练Loss: 5.6232 | 验证Loss: 5.6168 | LR: 1.50e-04 (x1.000)
生成样本： BA次窗aS題徘临處s轿之憂三湘其當一土t
Step    90 | 训练Loss: 5.6796 | 验证Loss: 5.7431 | LR: 7.50e-05 (x0.500)
生成样本： B莊A荷aS七言绝侶s虻七八珍居赴S秋然虞


In [7]:
from pathlib import Path

# 把训练结果保存下来，给后续训练使用
output_path = Path("./output/03_mid_train_model.pt")
output_path.parent.mkdir(parents=True, exist_ok=True)
torch.save(model.state_dict(), output_path)
print(f"模型已保存到 {output_path}")

模型已保存到 output/03_mid_train_model.pt
